In [1]:
from datasets import load_dataset as lds
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch
from tqdm import tqdm
import numpy as np
from dataset.loader import ContractNLIExample
import json

In [2]:
# Load dataset and model
dataset = ContractNLIExample.load(json.load(open('../dataset/contract-nli/dev.json','r')))
model_name = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


c:\Users\tanve\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

c:\Users\tanve\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tanve\.cache\huggingface\hub\models--microsoft--deberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

c:\Users\tanve\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\tanve\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\tanve\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
ds = lds("snli")
# Filter out examples with label -1
ds = ds.filter(lambda example: example['label'] != -1)

In [4]:
def tokenize_function(examples):
    # Tokenize the inputs
    tokenized = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        padding="max_length",
        truncation=True,
    )
    
    # Remap labels to match the model's expected label mapping
    label_remap = {0: 2, 1: 1, 2: 0}  # From labels to DeBERTa labels
    tokenized["labels"] = [label_remap[label] for label in examples["label"]]
    return tokenized

tokenized_ds = ds.map(tokenize_function, batched=True)
tokenized_ds.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

Map:   0%|          | 0/549367 [00:00<?, ? examples/s]

In [5]:
# Create DataLoader
test_dataset = tokenized_ds["test"]
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [6]:
def interpret_logits(logits):
    # Convert logits to CPU and extract values
    logits_cpu = logits.cpu().tolist()
    
    # If the first logit (index 0) is highest, it's entailment
    if logits_cpu[0] > logits_cpu[1] and logits_cpu[0] > logits_cpu[2]:
        return 0  # entailment
    # If the second logit is highest, it's neutral
    elif logits_cpu[1] > logits_cpu[0] and logits_cpu[1] > logits_cpu[2]:
        return 1  # neutral
    # Otherwise, it's contradiction
    else:
        return 2  # contradiction

In [7]:
def evaluate(model, dataloader, original_dataset):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    all_predictions = []
    all_labels = []
    
    # Model's label mapping
    model_label_map = {0: "contradiction", 1: "neutral", 2: "entailment"}
    # Original label mapping from the dataset
    original_label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Evaluating")
    
    with torch.no_grad():
        for batch_idx, batch in progress_bar:
            inputs = {
                'input_ids': batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device),
            }
            labels = batch['labels'].to(device)
            
            outputs = model(**inputs)
            logits = outputs.logits
            
            batch_preds = torch.argmax(logits, dim=-1)
            
            all_predictions.extend(batch_preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Calculate live metrics
            current_accuracy = accuracy_score(all_labels, all_predictions)
            current_f1 = f1_score(all_labels, all_predictions, average='weighted')
            
            progress_bar.set_postfix({'Accuracy': f'{current_accuracy:.4f}', 'F1': f'{current_f1:.4f}'})
            
            # Print examples for the first batch
            if batch_idx == 0:
                for j in range(min(10, len(batch['input_ids']))):
                    idx = batch_idx * dataloader.batch_size + j
                    if idx < len(original_dataset):
                        premise = original_dataset[idx]['premise']
                        hypothesis = original_dataset[idx]['hypothesis']
                        true_label_idx = original_dataset[idx]['label']
                        true_label = original_label_map[true_label_idx]
                        pred_label = model_label_map[batch_preds[j].item()]
                        
                        print(f"\nExample {idx + 1}:")
                        print(f"Premise: {premise}")
                        print(f"Hypothesis: {hypothesis}")
                        print(f"True label: {true_label}")
                        print(f"Predicted label: {pred_label}")
                        print(f"Logits: {logits[j].cpu().tolist()}")
                        print("-" * 50)
    
    # Calculate final metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    f1_macro = f1_score(all_labels, all_predictions, average='macro')
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
    
    print(f"\nFinal Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")
    
    return all_predictions, all_labels

In [8]:
# Run evaluation
predictions, true_labels = evaluate(model, test_dataloader, ds["test"])

Evaluating:   0%|          | 1/614 [00:05<52:34,  5.15s/it, Accuracy=0.3750, F1=0.2045]


Example 1:
Premise: This church choir sings to the masses as they sing joyous songs from the book at a church.
Hypothesis: The church has cracks in the ceiling.
True label: neutral
Predicted label: contradiction
Logits: [-0.045599281787872314, -0.10671716183423996]
--------------------------------------------------

Example 2:
Premise: This church choir sings to the masses as they sing joyous songs from the book at a church.
Hypothesis: The church is filled with song.
True label: entailment
Predicted label: contradiction
Logits: [-0.04570947587490082, -0.1068798080086708]
--------------------------------------------------

Example 3:
Premise: This church choir sings to the masses as they sing joyous songs from the book at a church.
Hypothesis: A choir singing at a baseball game.
True label: contradiction
Predicted label: contradiction
Logits: [-0.04625428467988968, -0.10744816064834595]
--------------------------------------------------

Example 4:
Premise: A woman with a green headsc

Evaluating:   7%|▋         | 43/614 [00:32<07:12,  1.32it/s, Accuracy=0.3285, F1=0.1624]


KeyboardInterrupt: 

In [ ]:
print(model)

In [ ]:
print(model.config)
